In [1]:
from pdf2image import convert_from_path

In [3]:
pages = convert_from_path("docs\patient_details\pd_1.pdf",
                          poppler_path=r"C:\Release-24.08.0-0\poppler-24.08.0\Library\bin")

In [4]:
# there are 2 pages 
pages

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1867x2000>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2000x1398>]

In [7]:
# information of our interest is in the 1st page 
pages[0].show()

In [8]:
import pytesseract

# converts image to text
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
text = pytesseract.image_to_string(pages[0],lang='eng')
print(text)

47/12/2020

Patient Medical Record

Patient Information Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight
9264 Ash Dr 95
New York City, 10005 .
United States Height:
190
In Case of Emergency
m _ a _
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History

Chicken Pox (Varicella):

IMMUNE

Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine



In [23]:
import numpy as np
import cv2

def preprocess_image(img):
    grey = cv2.cvtColor(np.array(img),cv2.COLOR_BGR2GRAY)
    resized_img = cv2.resize(grey,None,fx=2,fy=2,interpolation=cv2.INTER_LINEAR)
    processed_img = cv2.adaptiveThreshold(
    resized_img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,61,11
    )
    return processed_img

In [24]:
from PIL import Image
n_img = preprocess_image(pages[0])
Image.fromarray(n_img).show()

In [25]:
# Converting the image on which adaptive thresholding is applied we get all 
# the content in the image
txt = pytesseract.image_to_string(n_img,lang='eng')
print(txt)

17/12/2020

Patient Medical Record . : :

Patient Information

Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight:
9264 Ash Dr 95
New York City, 10005 a
United States Height:
190
In Case of Emergency
en oe
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History
I a
Chicken Pox (Varicella): Measies:
IMMUNE IMMUNE

Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine



In [29]:
# Extracting Patient name
import re

pattern = 'Patient Information(.*?)\(\d{3}\)'

matches = re.findall(pattern,txt,flags=re.DOTALL)
matches[0].strip()

'Birth Date\nKathy Crawford May 6 1972'

In [31]:
match = matches[0].replace("Birth Date","").strip()
match

'Kathy Crawford May 6 1972'

In [37]:
pattern = '((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
date_matches = re.findall(pattern,match)
date = date_matches[0][0]
date

'May 6 1972'

In [38]:
name = match.replace(date,"").strip()
name

'Kathy Crawford'

In [43]:
# extracts the name from the document
def ext_name(name):
    # remove birth date
    name = name.replace("Birth Date","").strip()
    # remove date
    date_pattern = '((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
    date_matches = re.findall(date_pattern,name)
    
    if date_matches:
        date = date_matches[0][0]
        name = name.replace(date,'').strip()
    return name

In [44]:
n = ext_name(matches[0].strip())
n

'Kathy Crawford'

In [45]:
nm = '\n\n \n\n \n\nBirth Date\nKathy Crawford May 6 1972\n'
nam = ext_name(nm)
nam

'Kathy Crawford'

In [49]:
# Extracting Phone
pattern = 'Patient Information(.*?)(\(\d{3}\) \d{3}-\d{4})'
matches = re.findall(pattern,txt,flags=re.DOTALL)
matches[0][1]

'(737) 988-0851'

In [50]:
# Extracting vaccine info

pattern = 'Have you had the Hepatitis B vaccination\?.*(Yes|No)'
matches = re.findall(pattern,txt,flags=re.DOTALL)
matches

['No']

In [53]:
# Extracting Medical issue info

pattern = 'List any Medical Problems \(asthma, seizures, headaches\):(.*)'
matches = re.findall(pattern,txt,flags=re.DOTALL)
matches[0].strip()

'Migraine'